In [1]:
# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
from typing import List

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes
from swin_unetr_seg_operator import SwinUnetrSegOperator
from unzip_operator import UnzipOperator

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.publisher_operator import PublisherOperator
from monai.deploy.operators.stl_conversion_operator import STLConversionOperator
import monai.deploy.core as md  

@resource(cpu=1, gpu=1, memory="7Gi")
@md.env(pip_packages=["gdcm"])
# pip_packages can be a string that is a path(str) to requirements.txt file or a list of packages.
# The MONAI pkg is not required by this class, instead by the included operators.
class AISwinUnetrSegApp(Application):
    def __init__(self, *args, **kwargs):
        """Creates an application instance."""

        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.debug(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.debug(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        self._logger.debug(f"Begin {self.compose.__name__}")
        # Creates the custom operator(s) as well as SDK built-in operator(s).
        unzip_op = UnzipOperator()
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Model specific inference operator, supporting MONAI transforms.
        swin_unetr_seg_op = SwinUnetrSegOperator()
        # Create the publisher operator
        publisher_op = PublisherOperator()
        # Create the surface mesh STL conversion operator, for all segments
        stl_conversion_op = STLConversionOperator(
            output_file="stl/multi-organs.stl", keep_largest_connected_component=False
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue.
        # The segment_label, algorithm_name, and algorithm_version are limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html

        _algorithm_name = "3D multi-organ segmentation from CT image"
        _algorithm_family = codes.DCM.ArtificialIntelligence
        _algorithm_version = "0.1.0"

        # List of (Segment name, [Code menaing str]), not including background which is value of 0.
        # User must provide correct codes, which can be looked at, e.g.
        # https://bioportal.bioontology.org/ontologies/SNOMEDCT
        # Alternatively, consult the concept and code dictionaries in PyDicom

        organs = [
            ("Spleen",),
            ("Right Kidney", "Kidney"),
            ("Left Kideny", "Kidney"),
            ("Gallbladder",),
            ("Esophagus",),
            ("Liver",),
            ("Stomach",),
            ("Aorta",),
            ("Inferior vena cava", "InferiorVenaCava"),
            ("Portal and Splenic Veins", "SplenicVein"),
            ("Pancreas",),
            ("Right adrenal gland", "AdrenalGland"),
            ("Left adrenal gland", "AdrenalGland"),
        ]

        segment_descriptions = [
            SegmentDescription(
                segment_label=organ[0],
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.__getattr__(organ[1] if len(organ) > 1 else organ[0]),
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            )
            for organ in organs
        ]

        dicom_seg_writer = DICOMSegmentationWriterOperator(segment_descriptions)

        # Create the processing pipeline, by specifying the source and destination operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        #self.add_flow(unzip_op,study_loader_op,{"unzip_folder": "dicom_study_list"})
        self.add_flow(unzip_op,study_loader_op)
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, swin_unetr_seg_op, {"image": "image"})
        #self.add_flow(unetr_seg_op, stl_conversion_op, {"seg_image": "image"})

        # Add the publishing operator to save the input and seg images for Render Server.
        # Note the PublisherOperator has temp impl till a proper rendering module is created.
        self.add_flow(swin_unetr_seg_op, publisher_op, {"saved_images_folder": "saved_images_folder"})

        # Note below the dicom_seg_writer requires two inputs, each coming from a source operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(swin_unetr_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

        self._logger.debug(f"End {self.compose.__name__}")

In [9]:
app = AISwinUnetrSegApp()

In [12]:
app.run(input="/home/jupyter/ct-seg-monai-swin-unetr/dcm", output="/home/jupyter/ct-seg-monai-swin-unetr/output", model="/home/jupyter/ct-seg-monai-swin-unetr/best_metric_model_gpu.ts")

Going to initiate execution of operator UnzipOperator
Executing operator UnzipOperator (Process ID: 43066, Operator ID: b8a5041b-56e7-4289-ac50-ab7bded62b9c)
/home/jupyter/ct-seg-monai-swin-unetr/dcm
/home/jupyter/ct-seg-monai-swin-unetr/dcm/1 - Axial Chest CT COVID-19.zip
Done performing execution of operator UnzipOperator

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 43066, Operator ID: f9a3a684-8a7a-4ca3-8a07-4d1416287feb)


[2023-08-07 16:24:17,191] [WARNING] (root) - No selection rules given; select all series.
[2023-08-07 16:24:17,191] [INFO] (root) - Working on study, instance UID: 1.2.826.0.1.3680043.2.1125.1.62837601195937447400039784641162299
[2023-08-07 16:24:17,192] [INFO] (root) - Working on series, instance UID: 1.2.826.0.1.3680043.2.1125.1.71161657915368769845404408463327121


Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 43066, Operator ID: cd547cb6-96b0-4e36-945a-4734bf6104c8)
Working on study, instance UID: 1.2.826.0.1.3680043.2.1125.1.62837601195937447400039784641162299
Working on series, instance UID: 1.2.826.0.1.3680043.2.1125.1.71161657915368769845404408463327121
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 43066, Operator ID: 0bbe7a5f-6193-4859-955f-8c58487b5089)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator SwinUnetrSegOperator
Executing operator SwinUnetrSegOperator (Process ID: 43066, Operator ID: 4b388539-217c-4f44-ae09-191abea955fe)
Operator output folder path: /home/jupyter/.monai_workdir/operat

[2023-08-07 16:24:42,842] [INFO] (highdicom.seg.sop) - add plane #0 for segment #1
[2023-08-07 16:24:42,845] [INFO] (highdicom.seg.sop) - add plane #1 for segment #1
[2023-08-07 16:24:42,847] [INFO] (highdicom.seg.sop) - add plane #2 for segment #1
[2023-08-07 16:24:42,849] [INFO] (highdicom.seg.sop) - add plane #3 for segment #1
[2023-08-07 16:24:42,851] [INFO] (highdicom.seg.sop) - add plane #4 for segment #1
[2023-08-07 16:24:42,853] [INFO] (highdicom.seg.sop) - add plane #5 for segment #1
[2023-08-07 16:24:42,854] [INFO] (highdicom.seg.sop) - add plane #6 for segment #1
[2023-08-07 16:24:42,856] [INFO] (highdicom.seg.sop) - add plane #7 for segment #1
[2023-08-07 16:24:42,859] [INFO] (highdicom.seg.sop) - add plane #8 for segment #1
[2023-08-07 16:24:42,860] [INFO] (highdicom.seg.sop) - add plane #9 for segment #1
[2023-08-07 16:24:42,862] [INFO] (highdicom.seg.sop) - add plane #10 for segment #1
[2023-08-07 16:24:42,864] [INFO] (highdicom.seg.sop) - add plane #11 for segment #1
[2

Done performing execution of operator DICOMSegmentationWriterOperator



In [15]:
!python /home/jupyter/ct-seg-monai-swin-unetr -i /home/jupyter/dcm -o /home/jupyter/output -m /home/jupyter/ct-seg-monai-swin-unetr/best_metric_model_gpu.ts

DEBUG:app.AISwinUnetrSegApp:Begin compose
DEBUG:app.AISwinUnetrSegApp:End compose
INFO:root:Start processing input in: /home/jupyter/dcm with results in: /home/jupyter/output
DEBUG:app.AISwinUnetrSegApp:Begin run
Going to initiate execution of operator UnzipOperator
Executing operator UnzipOperator (Process ID: 100527, Operator ID: ad422d53-2260-4dfd-8cf0-8edad4c2e8ed)
/home/jupyter/dcm
/home/jupyter/dcm/1 - Axial Chest CT COVID-19.zip
Done performing execution of operator UnzipOperator

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 100527, Operator ID: af574359-3e66-4f69-8fe4-02410c987fd4)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 100527, Operator ID: 635f2c42-ebd8-4d6b-9cdb-bdd0ffe80f0e)
[2023-08-07 17:32:08,024] [WARNING] (root) - No selection rules given; select 

In [3]:
!monai-deploy exec "/home/jupyter/ct-seg-monai-swin-unetr" -i "/home/jupyter/dcm" -o "/home/jupyter/output" -m "/home/jupyter/ct-seg-monai-swin-unetr/best_metric_model_gpu.ts"

DEBUG:app.AISwinUnetrSegApp:Begin compose
DEBUG:app.AISwinUnetrSegApp:End compose
INFO:root:Start processing input in: /home/jupyter/dcm with results in: /home/jupyter/output
DEBUG:app.AISwinUnetrSegApp:Begin run
Going to initiate execution of operator UnzipOperator
Executing operator UnzipOperator (Process ID: 5581, Operator ID: b67ee37b-f7a9-47b6-bb2e-38a031efc267)
/home/jupyter/dcm
/home/jupyter/dcm/1 - Axial Chest CT COVID-19.zip
Done performing execution of operator UnzipOperator

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 5581, Operator ID: 0d3a9aaf-4d6a-4561-9484-7c29f9386769)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 5581, Operator ID: bf0c969d-4247-4f6c-afdf-84286e2015f5)
[2023-08-07 20:18:59,951] [WARNING] (root) - No selection rules given; select all se

In [2]:
!monai-deploy exec "/home/jupyter/ct-seg-monai-swin-unetr" -i "/home/jupyter/dcm" -o "/home/jupyter/output" -m "/home/jupyter/ct-seg-monai-swin-unetr/best_metric_model.ts"

DEBUG:app.AISwinUnetrSegApp:Begin compose
DEBUG:app.AISwinUnetrSegApp:End compose
INFO:root:Start processing input in: /home/jupyter/dcm with results in: /home/jupyter/output
DEBUG:app.AISwinUnetrSegApp:Begin run
Going to initiate execution of operator UnzipOperator
Executing operator UnzipOperator (Process ID: 5387, Operator ID: 29ad07ad-897d-4d4b-9627-3aacbd94d5a1)
/home/jupyter/dcm
/home/jupyter/dcm/1 - Axial Chest CT COVID-19.zip
Done performing execution of operator UnzipOperator

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 5387, Operator ID: 1fec72b2-71e1-4c4a-a24e-463cb92084fd)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 5387, Operator ID: dba4aa62-ace7-456e-8773-dbabcee8dd57)
[2023-08-07 20:17:46,163] [WARNING] (root) - No selection rules given; select all se

In [4]:
%cd ..

/home/jupyter


In [23]:
!monai-deploy package -b nvcr.io/nvidia/pytorch:23.07-py3 /home/jupyter/ct-seg-monai-swin-unetr --tag flywheel/ct-seg-monai-swin-unetr:0.1.5 -m /home/jupyter/ct-seg-monai-swin-unetr/best_metric_model_gpu.ts

Building MONAI Application Package... Done
[2023-08-07 19:51:23,495] [INFO] (app_packager) - Successfully built flywheel/ct-seg-monai-swin-unetr:0.1.5


In [ ]:
# this image can run for now
'''
docker run -it -v /home/jupyter/dcm:/var/monai/input --entrypoint=/bin/bash flywheel/ct-seg-monai-swin-unetr-cpu:0.1.6
python "/opt/monai/app" -i "/var/monai/input" -o "/var/monai/output" -m "/opt/monai/app/best_metric_model.ts"
'''

In [8]:
!monai-deploy package -b nvcr.io/nvidia/pytorch:23.07-py3 /home/jupyter/ct-seg-monai-swin-unetr --tag flywheel/ct-seg-monai-swin-unetr-cpu:0.1.6 -m /home/jupyter/ct-seg-monai-swin-unetr/best_metric_model.ts

Building MONAI Application Package... Done
[2023-08-07 21:20:42,937] [INFO] (app_packager) - Successfully built flywheel/ct-seg-monai-swin-unetr-cpu:0.1.6


In [7]:
!monai-deploy run flywheel/ct-seg-monai-swin-unetr-cpu:0.1.5 /home/jupyter/dcm /home/jupyter/output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "flywheel/ct-seg-monai-swin-unetr-cpu:0.1.5" is available...

Checking for MAP "flywheel/ct-seg-monai-swin-unetr-cpu:0.1.5" locally
"flywheel/ct-seg-monai-swin-unetr-cpu:0.1.5" found.

Reading MONAI App Package manifest...
--> Verifying if "nvidia-docker" is installed...

DEBUG:__main__.AISwinUnetrSegApp:Begin compose
DEBUG:__main__.AISwinUnetrSegApp:End compose
INFO:root:Start processing input in: /var/monai/input/1 - Axial Chest CT COVID-19 with results in: /var/monai/output/1 - Axial Chest CT COVID-19_output
DEBUG:__main__.AISwinUnetrSegApp:Begin run
Going to initiate execution of operator UnzipOperator
Executing operator UnzipOperator (Process ID: 1, Operator ID: 55d3a3d8-0517-4a75-96b5-42d6ab2520ca)
/var/monai/input/1 - Axial Chest CT COVID-19
[2023-08-07 20:46:21,936] [ERROR] (root) - Failed processing input in /var/monai/input/1 - Axial Chest CT COVID-19, due to: 

Traceback (most recent call la

In [5]:
!monai-deploy run flywheel/ct-seg-monai-swin-unetr:0.1.5 /home/jupyter/dcm /home/jupyter/output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "flywheel/ct-seg-monai-swin-unetr:0.1.5" is available...

Checking for MAP "flywheel/ct-seg-monai-swin-unetr:0.1.5" locally
"flywheel/ct-seg-monai-swin-unetr:0.1.5" found.

Reading MONAI App Package manifest...
--> Verifying if "nvidia-docker" is installed...

DEBUG:__main__.AISwinUnetrSegApp:Begin compose
DEBUG:__main__.AISwinUnetrSegApp:End compose
INFO:root:Start processing input in: /var/monai/input/1 - Axial Chest CT COVID-19 with results in: /var/monai/output/1 - Axial Chest CT COVID-19_output
DEBUG:__main__.AISwinUnetrSegApp:Begin run
Going to initiate execution of operator UnzipOperator
Executing operator UnzipOperator (Process ID: 1, Operator ID: 22ae7e80-01f0-425c-943b-690d576f1916)
/var/monai/input/1 - Axial Chest CT COVID-19
[2023-08-07 20:24:43,671] [ERROR] (root) - Failed processing input in /var/monai/input/1 - Axial Chest CT COVID-19, due to: 

Traceback (most recent call last):
  File 

In [ ]:
!monai-deploy run flywheel/ct-seg-monai-swin-unetr:0.1.5 /home/jupyter/dcm /home/jupyter/output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "flywheel/ct-seg-monai-swin-unetr:0.1.4" is available...

Checking for MAP "flywheel/ct-seg-monai-swin-unetr:0.1.4" locally
"flywheel/ct-seg-monai-swin-unetr:0.1.4" found.

Reading MONAI App Package manifest...
--> Verifying if "nvidia-docker" is installed...

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
DEBUG:__main__.AISwinUnetrSegApp:Begin compose
DEBUG:__main__.AISwinUnetrSegApp:End compose
